<a href="https://www.bigdatauniversity.com"><img src = "https://ibm.box.com/shared/static/cw2c7r3o20w9zn8gkecaeyjhgw3xdgbj.png" width = 400, align = "center"></a>

# <center>Decision Trees</center>

In this lab exercise, you will learn a popular machine learning algorithm, Decision Tree. You will use this classification algorithm to build a model from historical data of patients, and their respond to different medications. Then you use the trained decision tree to predict the class of a unknown patient, or to find a proper drug for a new patient.

Import the Following Libraries:
<ul>
    <li> <b>numpy (as np)</b> </li>
    <li> <b>pandas</b> </li>
    <li> <b>DecisionTreeClassifier</b> from <b>sklearn.tree</b> </li>
</ul>

In [70]:
import numpy as np 
import pandas as pd
from sklearn.tree import DecisionTreeClassifier

### About dataset
Imagine that you are a medical researcher compiling data for a study. You have collected data about a set of patients, all of whom suffered from the same illness. During their course of treatment, each patient responded to one of 5 medications, Drug A, Drug B, Drug c, Drug x and y. 

Part of your job is to build a model to find out which drug might be appropriate for a future patient with the same illness. The feature sets of this dataset are Age, Sex, Blood Pressure, and Cholesterol of patients, and the target is the drug that each patient responded to. 

It is a sample of binary classifier, and you can use the training part of the dataset 
to build a decision tree, and then use it to predict the class of a unknown patient, or to prescribe it to a new patient.


### Downloading Data
To download the data, we will use !wget to download it from IBM Object Storage.

__Did you know?__ When it comes to Machine Learning, you will likely be working with large datasets. As a business, where can you host your data? IBM is offering a unique opportunity for businesses, with 10 Tb of IBM Cloud Object Storage: [Sign up now for free](http://cocl.us/ML0101EN-IBM-Offer-CC)

now, read data using pandas dataframe:

In [71]:
df = pd.read_csv("stroke.csv")
df[0:5]

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,9046,Male,67.0,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked,1
1,51676,Female,61.0,0,0,Yes,Self-employed,Rural,202.21,NaN,never smoked,1
2,31112,Male,80.0,0,1,Yes,Private,Rural,105.92,32.5,never smoked,1
3,60182,Female,49.0,0,0,Yes,Private,Urban,171.23,34.4,smokes,1
4,1665,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.0,never smoked,1


## Practice 
What is the size of data? 

In [72]:
# write your code here

avg_norm_loss = df['bmi'].astype("float").mean(axis=0)
df['bmi'].replace(np.nan, avg_norm_loss, inplace=True)


## Pre-processing

Using <b>my_data</b> as the Drug.csv data read by pandas, declare the following variables: <br>
<ul>
    <li> <b> X </b> as the <b> Feature Matrix </b> (data of my_data) </li>

    
    <li> <b> y </b> as the <b> response vector (target) </b> </li>


   
</ul>

Remove the column containing the target name since it doesn't contain numeric values.

In [73]:

X = df[['gender','age','hypertension','ever_married','work_type','Residence_type','heart_disease','avg_glucose_level','bmi','smoking_status','stroke']].values
print(X[0:5])

[['Male' 67.0 0 'Yes' 'Private' 'Urban' 1 228.69 36.6 'formerly smoked' 1]
 ['Female' 61.0 0 'Yes' 'Self-employed' 'Rural' 0 202.21
  28.893236911794673 'never smoked' 1]
 ['Male' 80.0 0 'Yes' 'Private' 'Rural' 1 105.92 32.5 'never smoked' 1]
 ['Female' 49.0 0 'Yes' 'Private' 'Urban' 0 171.23 34.4 'smokes' 1]
 ['Female' 79.0 1 'Yes' 'Self-employed' 'Rural' 0 174.12 24.0
  'never smoked' 1]]


As you may figure out, some featurs in this dataset are catergorical such as __Sex__ or __BP__. Unfortunately, Sklearn Decision Trees do not handle categorical variables. But still we can convert these features to numerical values. __pandas.get_dummies()__
Convert categorical variable into dummy/indicator variables.

In [74]:
from sklearn import preprocessing

sex = preprocessing.LabelEncoder()
sex.fit(['Male','Female','Other'])
X[:,0] = sex.transform(X[:,0]) 


married = preprocessing.LabelEncoder()
married.fit([ 'Yes', 'No'])
X[:,3] = married.transform(X[:,3])


work = preprocessing.LabelEncoder()
work.fit([ 'Private', 'Self-employed','Govt_job','children','Never_worked'])
X[:,4] = work.transform(X[:,4]) 

residence = preprocessing.LabelEncoder()
residence.fit([ 'Urban','Rural'])
X[:,5] = residence.transform(X[:,5]) 

smoke = preprocessing.LabelEncoder()
smoke.fit([ 'formerly smoked','never smoked','smokes','Unknown'])
X[:,9] = smoke.transform(X[:,9]) 

X[0:5]


array([[1, 67.0, 0, 1, 2, 1, 1, 228.69, 36.6, 1, 1],
       [0, 61.0, 0, 1, 3, 0, 0, 202.21, 28.893236911794673, 2, 1],
       [1, 80.0, 0, 1, 2, 0, 1, 105.92, 32.5, 2, 1],
       [0, 49.0, 0, 1, 2, 1, 0, 171.23, 34.4, 3, 1],
       [0, 79.0, 1, 1, 3, 0, 0, 174.12, 24.0, 2, 1]], dtype=object)

Now we can fill the target variable.

In [75]:
y = df['stroke']
print(y[0:5])

0    1
1    1
2    1
3    1
4    1
Name: stroke, dtype: int64


---
## Setting up the Decision Tree
We will be using <b>train/test split</b> on our <b>decision tree</b>. Let's import <b>train_test_split</b> from <b>sklearn.cross_validation</b>.

In [76]:
from sklearn.model_selection import train_test_split

Now <b> train_test_split </b> will return 4 different parameters. We will name them:<br>
X_trainset, X_testset, y_trainset, y_testset <br> <br>
The <b> train_test_split </b> will need the parameters: <br>
X, y, test_size=0.3, and random_state=3. <br> <br>
The <b>X</b> and <b>y</b> are the arrays required before the split, the <b>test_size</b> represents the ratio of the testing dataset, and the <b>random_state</b> ensures that we obtain the same splits.

In [77]:
X_trainset, X_testset, y_trainset, y_testset = train_test_split(X, y, test_size=0.3, random_state=3)

## Practice
Print the shape of X_trainset and y_trainset. Ensure that the dimensions match

In [78]:
# your code
print('Train set:', X_trainset, y_trainset)
print('Test set:', X_testset, y_testset)


Train set: [[0 17.0 0 ... 30.3 2 0]
 [1 27.0 0 ... 29.5 3 0]
 [0 45.0 0 ... 28.0 2 0]
 ...
 [0 40.0 0 ... 27.9 3 0]
 [1 1.88 0 ... 13.5 0 0]
 [0 32.0 0 ... 44.8 2 0]] 3160    0
2730    0
1861    0
1825    0
510     0
       ..
789     0
968     0
1667    0
3321    0
1688    0
Name: stroke, Length: 3577, dtype: int64
Test set: [[0 29.0 0 ... 23.4 2 0]
 [1 58.0 0 ... 34.5 2 0]
 [0 37.0 0 ... 39.4 2 0]
 ...
 [0 34.0 0 ... 27.5 0 0]
 [0 41.0 0 ... 25.5 2 0]
 [0 69.0 1 ... 28.893236911794673 2 0]] 2778    0
4029    0
261     0
1868    0
1028    0
       ..
2794    0
1432    0
2212    0
2824    0
2343    0
Name: stroke, Length: 1533, dtype: int64


Print the shape of X_testset and y_testset. Ensure that the dimensions match

In [79]:
# your code



## Modeling
We will first create an instance of the <b>DecisionTreeClassifier</b> called <b>drugTree</b>.<br>
Inside of the classifier, specify <i> criterion="entropy" </i> so we can see the information gain of each node.

In [80]:
drugTree = DecisionTreeClassifier(criterion="entropy", max_depth = 4)
drugTree # it shows the default parameters

DecisionTreeClassifier(criterion='entropy', max_depth=4)

Next, we will fit the data with the training feature matrix <b> X_trainset </b> and training  response vector <b> y_trainset </b>

In [81]:
drugTree.fit(X_trainset,y_trainset)

DecisionTreeClassifier(criterion='entropy', max_depth=4)

## Prediction
Let's make some <b>predictions</b> on the testing dataset and store it into a variable called <b>predTree</b>.

In [82]:
predictTree = drugTree.predict(X_testset)

You can print out <b>predTree</b> and <b>y_testset</b> if you want to visually compare the prediction to the actual values.

In [83]:
print (predictTree [0:5])
print (y_testset [0:5])


[0 0 0 0 0]
2778    0
4029    0
261     0
1868    0
1028    0
Name: stroke, dtype: int64


## Evaluation
Next, let's import __metrics__ from sklearn and check the accuracy of our model.

In [84]:
from sklearn import metrics
import matplotlib.pyplot as plt
print("DecisionTrees's Accuracy: ", metrics.accuracy_score(y_testset, predictTree))

DecisionTrees's Accuracy:  1.0


__Accuracy classification score__ computes subset accuracy: the set of labels predicted for a sample must exactly match the corresponding set of labels in y_true.  

In multilabel classification, the function returns the subset accuracy. If the entire set of predicted labels for a sample strictly match with the true set of labels, then the subset accuracy is 1.0; otherwise it is 0.0.


## Practice 
Can you calculate the accuracy score without sklearn ?

In [85]:
# your code here



## Visualization
Lets visualize the tree

## Want to learn more?

IBM SPSS Modeler is a comprehensive analytics platform that has many machine learning algorithms. It has been designed to bring predictive intelligence to decisions made by individuals, by groups, by systems – by your enterprise as a whole. A free trial is available through this course, available here: [SPSS Modeler](http://cocl.us/ML0101EN-SPSSModeler).

Also, you can use Watson Studio to run these notebooks faster with bigger datasets. Watson Studio is IBM's leading cloud solution for data scientists, built by data scientists. With Jupyter notebooks, RStudio, Apache Spark and popular libraries pre-packaged in the cloud, Watson Studio enables data scientists to collaborate on their projects without having to install anything. Join the fast-growing community of Watson Studio users today with a free account at [Watson Studio](https://cocl.us/ML0101EN_DSX)

### Thanks for completing this lesson!

Notebook created by: <a href = "https://ca.linkedin.com/in/saeedaghabozorgi">Saeed Aghabozorgi</a>

<hr>
Copyright &copy; 2018 [Cognitive Class](https://cocl.us/DX0108EN_CC). This notebook and its source code are released under the terms of the [MIT License](https://bigdatauniversity.com/mit-license/).​